In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import utils
import pandas as pd
import re
from parameters import labels, oh_labels, params

Main module did not execute


In [2]:
# Reorder the labels by taking into account the alphanumeric file order of the images.
# Needed preprocessing step to use tf.data.Dataset.
labels_list = utils.reorder_image_labels(params['image_path'], labels.iloc[:, 0].tolist(), index_start=1)

In [3]:
def str_to_numb(string_list):
    """

    :param string_list: list with of all the image numbers
    :return: numbers missing
    """
    number_list = []
    for string in string_list:
        number = re.findall(r'\d+', string)
        number_list.append(int(number[0].strip()))
    # number_list.sort()
    return number_list

In [4]:
image_paths = os.listdir(params['image_path'])
image_paths.sort(key=len)
img_numbers = str_to_numb(image_paths)
img_paths = []
j = 0
gegen = j + 1
for image in image_paths:
    if img_numbers[j] == gegen:
        img_paths.append(os.path.join(params['image_path'], image))
        gegen += 1
    else:
        print(f"Warning: picture {j + 1} is missing from the dataset")
        img_paths.append('None')
        img_paths.append(os.path.join(params['image_path'], image))
        gegen += 2
    j += 1

df = pd.DataFrame()
df['Image Paths'] = img_paths

In [5]:
images = df['Image Paths'].values

In [6]:
base_model = tf.keras.applications.resnet.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=params['IMG_SHAPE'],
)
base_model.trainable = params['TRAINABLE']
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  175


In [7]:
if params['TRAINABLE'] and params['TUNE_LAYER'] is not None:
    # Freeze all the layers before the `fine_tune_at` layer
    for layer in params['TUNE_LAYER']:
      layer.trainable = False

In [12]:
def create_model(img_shape, class_num, lr=1e-5, metrics=None):

    preprocess_input = tf.keras.applications.resnet.preprocess_input

    inputs = tf.keras.Input(shape=img_shape)
    x = preprocess_input(inputs)
    x = base_model(x, training=params['TRAINABLE'])
    # x = tf.keras.layers.Dropout(params['DROPOUT'])(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(params['DROPOUT'])(x)
    # x = tf.keras.layers.Dense(512, activation='relu')(x)
    # x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(class_num, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=metrics)

    return model

def preprocess_dataset(imagepath, entries, labels, image_type='jpg', search_type='*', cache=True, batch_size=8, shuffle=True, augment=False, augmentation=None, prefetch=True):

    dataset = utils.ImageDataset(imagepath, entries=entries, labels=labels, image_type=image_type, search_type=search_type)
    dataset = dataset.create_image_dataset()
    dataset = utils.ImageDataset.performance_config(dataset, cache=cache, batch_size=batch_size, shuffle=shuffle, augment=augment, augmentation=augmentation, prefetch=prefetch)

    return dataset

def evaluate_image(model, imagepath, target_size=None):

    img = tf.keras.utils.load_img(imagepath, target_size=target_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array, verbose=0)
    score = tf.nn.softmax(predictions[0])

    return np.argmax(score), np.max(score)

In [15]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                              min_delta=1e-2,
                                              patience=5,
                                              verbose=1),
             tf.keras.callbacks.ModelCheckpoint(params['save_path'] + 'model/',
                                      save_best_only=True,
                                      verbose=0,
                                      save_weights_only=True),
             tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                  factor=0.1,
                                                  patience=3,
                                                  min_delta=1e-2)
            ]

data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                                         tf.keras.layers.RandomRotation(0.5),
                                         tf.keras.layers.RandomZoom(0.1),
                                         tf.keras.layers.RandomContrast(0.3),
                                         tf.keras.layers.RandomBrightness(0.2)])

In [10]:
oh_list_labels = tf.keras.utils.to_categorical(labels_list, params['CLASS_NUMBER'])

In [16]:
if params['KFOLD']:
    # split the data to dev and test set
    outer_skf = utils.split_data(df,
                           labels_list,
                           split_strategy='StratifiedKFold',
                           n_splits=params['OUTER_SPLITS'],
                           shuffle=params['shuffle'],
                           random_state=None) #params['SEED'])
    # create a Logger object to record and display the values we need
    log = utils.Logger()
    experiment = 1
    # log the parameters used to json
    log.to_json(params['save_path'] + f'Experiment_{experiment}_parameters.json', params)
    model_history = {}
    # ensure use of gpu
    with tf.device("gpu:0"):

        for (dev_idx, test_idx), i in outer_skf:

            print(f"\nStarting Training on Outer Fold {i+1}/{params['OUTER_SPLITS']}.")
            print("===========================================")
            # create the inner splits
            #labels_array = np.array(labels_list)[dev_idx.astype(int)]

            inner_skf = utils.split_data(df.loc[dev_idx, :],
                                   np.array(labels_list)[dev_idx.astype(int)],
                                   split_strategy='StratifiedKFold',
                                   n_splits=params['INNER_SPLITS'],
                                   shuffle=params['shuffle'],
                                   random_state=None) #params['SEED'])

            fold_history = []

            model_history[f"Fold_{i}_acc"] = []

            test = preprocess_dataset(params['image_path'],
                             images[test_idx],
                             oh_list_labels[test_idx, :],
                             batch_size=params['BATCH_SIZE'],
                             shuffle=params['shuffle'],
                             cache=params['cache'])

            for (train_idx, val_idx), j in inner_skf:


                train = preprocess_dataset(params['image_path'],
                               images[train_idx],
                               oh_list_labels[train_idx, :],
                               batch_size=params['BATCH_SIZE'],
                               cache=params['cache'],
                               shuffle=params['shuffle'],
                               augment=params['augment'],
                               augmentation=data_augmentation)

                val = preprocess_dataset(params['image_path'],
                             images[val_idx],
                             oh_list_labels[val_idx, :],
                             batch_size=params['BATCH_SIZE'],
                             shuffle=params['shuffle'],
                             cache=params['cache'])

                model = create_model(params['IMG_SHAPE'], params['CLASS_NUMBER'], params['LR'], metrics=params['metrics'])


                print(f"\nStarting Training on Inner Fold {j+1}/{params['INNER_SPLITS']}.")
                print("===========================================")

                fold_hist = model.fit(train,
                                      epochs=params['MAX_EPOCHS'],
                                      validation_data=val,
                                      callbacks=callbacks)


                # create a list of the metrics names used
                metrics = ["val_" + metric.lower() for metric in params['metrics']]
                # create a list of the metrics values
                metrics_values = [np.amax(fold_hist.history[metric]) for metric in metrics]
                # add the metrics and their labels to a dictionary
                fold_metrics = {key:val for key, val in zip(metrics, metrics_values ) }
                # Log the train metrics to an excel file and retrieve the experiment number
                experiment = log.to_excel(params['save_path'] + 'results.xlsx', fold_metrics)
                # change the type of the training metrics to float so they can be serialized in json
                model_history[f"Fold_{i}.{j}_hist"] = {key: list(map(float, val)) for key, val in fold_hist.history.items()}

                model_history[f"Fold_{i}.{j}_acc"] = []

                model_history[f"Fold_{i}.{j}_test"] = dict(zip(model.metrics_names, model.evaluate(test)))


                print(f"          Fold {i}.{j} Results")
                print("**|" + "-"* 30 + "|**")
                # m = tf.keras.metrics.Accuracy()
                # m.update_state(model_history[f"Fold_{i}.{j}_preds"], labels.loc[test_idx, :])
                accuracy = model_history[f"Fold_{i}.{j}_test"]['categorical_accuracy']
                model_history[f"Fold_{i}.{j}_acc"].append(accuracy)
                print(f"  |       Accuracy = {accuracy:.3f}       |")


            acc_avg = np.mean(model_history[f"Fold_{i}.{j}_acc"])
            acc_std = np.std(model_history[f"Fold_{i}.{j}_acc"])

            model_history[f"Fold_{i}_acc"] = acc_avg
            model_history[f"Fold_{i}_std"] = model_history[f"Fold_{i}.{j}_acc"]

            print(f"\n            Fold {i} Averaged Results")
            print("**|" + "-"* 40 + "|**")
            print(f"  |          Mean Accuracy = {acc_avg:.3f}         |")
            print(f"  |      Mean Standard Deviation = {acc_std:.3f}   |")

    # Log the model training history to json
    log.to_json(params['save_path'] + f'Experiment_{experiment}_history.json', model_history)


Starting Training on Outer Fold 1/20.

Starting Training on Inner Fold 1/5.
Epoch 1/100
10/10 [==============================] - 7s 359ms/step - loss: 2.3737 - categorical_accuracy: 0.1972 - precision: 0.1856 - recall: 0.1659 - auc: 0.3499 - val_loss: 2.0211 - val_categorical_accuracy: 0.1125 - val_precision: 0.0598 - val_recall: 0.0437 - val_auc: 0.2197 - lr: 1.0000e-05
Epoch 2/100
10/10 [==============================] - 2s 167ms/step - loss: 2.0846 - categorical_accuracy: 0.2551 - precision: 0.2629 - recall: 0.2238 - auc: 0.4224 - val_loss: 1.8660 - val_categorical_accuracy: 0.1375 - val_precision: 0.1062 - val_recall: 0.0750 - val_auc: 0.2693 - lr: 1.0000e-05
Epoch 3/100
10/10 [==============================] - 2s 174ms/step - loss: 1.9966 - categorical_accuracy: 0.2613 - precision: 0.2559 - recall: 0.2222 - auc: 0.4270 - val_loss: 1.7172 - val_categorical_accuracy: 0.1875 - val_precision: 0.1296 - val_recall: 0.0875 - val_auc: 0.3240 - lr: 1.0000e-05
Epoch 4/100
10/10 [==========

In [17]:
loss, categorical_accuracy, precision, recall, auc = model.evaluate(test)

1/1 [==============================] - 0s 295ms/step - loss: 0.1264 - categorical_accuracy: 0.9762 - precision: 0.9756 - recall: 0.9524 - auc: 0.9994
